<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True, dtype=str)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
"""
Import Statements
"""

# Base
from collections import Counter
import re

import pandas as pd

# Plotting
import squarify
import matplotlib.pyplot as plt
import seaborn as sns

# NLP Libraries
import spacy
from spacy.tokenizer import Tokenizer
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize # Sentence Tokenizer
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

nlp = spacy.load("en_core_web_lg")

[nltk_data] Downloading package punkt to /home/seek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/seek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [5]:
yelp['tokens'] = yelp['text'].apply(tokenize)

In [6]:
yelp_tokens = yelp['tokens']

yelp_tokens.head(20)

0     [beware, fake, fake, fake, small, business, lo...
1     [came, lunch, togo, service, quick, staff, fri...
2     [ve, vegas, dozens, times, stepped, foot, circ...
3     [went, night, closed, street, party, best, act...
4     [stars, bad, price, lunch, seniors, pay, eatin...
5     [tasty, fast, casual, latin, street, food, men...
6     [absolutely, amazing, incredible, production, ...
7     [came, pho, enjoyed, got, pm, busy, got, serve...
8     [absolutely, unique, experience, nail, shop, f...
9     [wow, walked, sat, bar, minutes, bartenders, w...
10    [popped, dinner, yesterday, reservation, despi...
11    [thw, worst, stay, ended, paying, nights, said...
12    [great, friendly, customer, service, quality, ...
13    [food, great, super, busy, server, attentive, ...
14    [talk, getting, ripped, charged, shut, valve, ...
15    [girls, night, tonight, kid, decided, drive, h...
16    [stopped, drinks, flying, charlotte, weeks, ba...
17    [excellent, restaurant, encourage, visitin

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [7]:
data = yelp_tokens

In [8]:
yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
business_id    10000 non-null object
cool           10000 non-null object
date           10000 non-null object
funny          10000 non-null object
review_id      10000 non-null object
stars          10000 non-null object
text           10000 non-null object
useful         10000 non-null object
user_id        10000 non-null object
tokens         10000 non-null object
dtypes: object(10)
memory usage: 781.3+ KB


In [9]:
yelp_tokens = yelp['tokens'].astype(str)

In [10]:
data = yelp_tokens

In [14]:
# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

# Create a vocabulary and get word counts per document
dtm = tfidf.fit_transform(data)
#dtm = tfidf.transform(new_data)

# Print word counts

# Get feature names to use as dataframe column headers

# View Feature Matrix as DataFrame
docs = pd.DataFrame(dtm.todense(), columns = tfidf.get_feature_names())
docs.head()

,ability,able,absolute,absolutely,ac,accent,accept,acceptable,accepted,access,...,yuck,yuk,yum,yummy,yup,zero,zone,zoo,zucchini,était
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Get Word Counts for each document
dtm.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.08374961, ..., 0.        , 0.        ,
         0.        ]])

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

dist_matrix  = cosine_similarity(dtm)

In [17]:
dist_df = pd.DataFrame(dist_matrix)

dist_df[0].idxmax(axis=0)

0

In [18]:
dist_df.shape

(10000, 10000)

In [19]:
dist_df[0]

0       1.000000
1       0.000000
2       0.014769
3       0.000000
4       0.000000
5       0.028595
6       0.003441
7       0.005949
8       0.002335
9       0.033934
10      0.008032
11      0.044258
12      0.000000
13      0.000000
14      0.161921
15      0.000000
16      0.003988
17      0.000000
18      0.020421
19      0.020425
20      0.022786
21      0.003015
22      0.070319
23      0.003713
24      0.000000
25      0.012260
26      0.000000
27      0.000000
28      0.000000
29      0.007559
          ...   
9970    0.031191
9971    0.025284
9972    0.000000
9973    0.065152
9974    0.000000
9975    0.002760
9976    0.023955
9977    0.000000
9978    0.000000
9979    0.006521
9980    0.004137
9981    0.000000
9982    0.022165
9983    0.035718
9984    0.005287
9985    0.018542
9986    0.022124
9987    0.000000
9988    0.011291
9989    0.006770
9990    0.000000
9991    0.000000
9992    0.000000
9993    0.013500
9994    0.022723
9995    0.055663
9996    0.000000
9997    0.0000

In [20]:
# Instantiate
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
# Fit on TF-IDF Vectors
nn.fit(dtm.todense())

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [21]:
# Query Using kneighbors
nn.kneighbors(dtm.todense()[0])

(array([[0.        , 1.        , 1.        , 1.25098799, 1.25476368]]),
 array([[   0, 6204, 6311, 2131, 3276]]))

In [26]:
fake_review = ["""This place is a disaster. DO NOT EVER GO TO THIS PLACE!!!!"""]

In [27]:
new = tfidf.transform(fake_review)

nn.kneighbors(new.todense())

(array([[1.        , 1.        , 1.1859987 , 1.19196279, 1.22612331]]),
 array([[6204, 6311, 6338, 5129, 8191]]))

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)